## ETL - Extract Transform Load
Методика ETL подразумевает под собой получение, очистку и объединение входных данных для дальнейшего анализа

## Загрузка библиотек

In [40]:
import pandas as pd
import numpy as np

# visualization
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 16, 10

## Загрузка и просмотр данных

In [22]:
# данные о зданиях
buildings_data = pd.read_csv('../data/building_metadata.csv.gz')
buildings_data.info()
buildings_data.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site_id      1449 non-null   int64  
 1   building_id  1449 non-null   int64  
 2   primary_use  1449 non-null   object 
 3   square_feet  1449 non-null   int64  
 4   year_built   675 non-null    float64
 5   floor_count  355 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.0+ KB


,site_id,building_id,primary_use,square_feet,year_built,floor_count
916,9,916,Education,51778,NaN,NaN
980,9,980,Entertainment/public assembly,34819,NaN,NaN
1388,15,1388,Healthcare,45465,1990.0,NaN


In [23]:
# данные о погодных условиях
weather_data = pd.read_csv('../data/weather_train.csv.gz')
weather_data.info()
weather_data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             139773 non-null  int64  
 1   timestamp           139773 non-null  object 
 2   air_temperature     139718 non-null  float64
 3   cloud_coverage      70600 non-null   float64
 4   dew_temperature     139660 non-null  float64
 5   precip_depth_1_hr   89484 non-null   float64
 6   sea_level_pressure  129155 non-null  float64
 7   wind_direction      133505 non-null  float64
 8   wind_speed          139469 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0


In [24]:
# данные об энергопотреблении здания с индексом 0
energy0_data = pd.read_csv('../data/train.0.0.csv.gz')
energy0_data.info()
energy0_data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   building_id    8784 non-null   int64  
 1   meter          8784 non-null   int64  
 2   timestamp      8784 non-null   object 
 3   meter_reading  8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 274.6+ KB


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,0,0,2016-01-01 01:00:00,0.0
2,0,0,2016-01-01 02:00:00,0.0


## Объединение данных

In [51]:
# энергопотребление + данные о здании с индексом 0
result_data = energy0_data.merge(buildings_data, on='building_id', how='left')
result_data = result_data.merge(weather_data, on=['site_id', 'timestamp'], how='left')
result_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 8783
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   building_id         8784 non-null   int64  
 1   meter               8784 non-null   int64  
 2   timestamp           8784 non-null   object 
 3   meter_reading       8784 non-null   float64
 4   site_id             8784 non-null   int64  
 5   primary_use         8784 non-null   object 
 6   square_feet         8784 non-null   int64  
 7   year_built          8784 non-null   float64
 8   floor_count         0 non-null      float64
 9   air_temperature     8781 non-null   float64
 10  cloud_coverage      4954 non-null   float64
 11  dew_temperature     8781 non-null   float64
 12  precip_depth_1_hr   8783 non-null   float64
 13  sea_level_pressure  8699 non-null   float64
 14  wind_direction      8534 non-null   float64
 15  wind_speed          8784 non-null   float64
dtypes: flo

## Нахождение пропущенных значений и их замена

In [52]:
for col in result_data.columns:
    number_of_nan = result_data[col].isna().sum()
    if number_of_nan > 0:
        percent = np.round(result_data[col].isna().sum()/len(result_data), 6)
        print('column <{}>  {}% null-values'.format(col, percent))

column <floor_count>  1.0% null-values
column <air_temperature>  0.000342% null-values
column <cloud_coverage>  0.43602% null-values
column <dew_temperature>  0.000342% null-values
column <precip_depth_1_hr>  0.000114% null-values
column <sea_level_pressure>  0.009677% null-values
column <wind_direction>  0.028461% null-values


In [53]:
result_data.drop('floor_count', axis=1, inplace=True)

result_data['wind_direction'] = result_data['wind_direction'].apply(lambda x: result_data['wind_direction'].mean() if x!=x else x)
result_data['sea_level_pressure'] = result_data['sea_level_pressure'].apply(lambda x: result_data['sea_level_pressure'].mean() if x!=x else x)
result_data['cloud_coverage'].fillna(0, inplace=True)
result_data['air_temperature'].fillna(0, inplace=True)
result_data['dew_temperature'].fillna(0, inplace=True)
result_data['precip_depth_1_hr'] = result_data['precip_depth_1_hr'].apply(lambda x: x if x>0 else 0)

result_data.isna().sum()

building_id           0
meter                 0
timestamp             0
meter_reading         0
site_id               0
primary_use           0
square_feet           0
year_built            0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64